In [58]:
import pandas as pd
import numpy as np
import os
import re
import random

In [59]:
import pandas as pd

# Specify the file path
file_path = r"/content/aws_review_sofware_dataset.csv"
# Initialize an empty DataFrame to store the first 1000 rows
df_5000_rows = pd.DataFrame()

# Read the dataset in chunks of, for example, 500 rows
chunksize = 5000

# Iterate over chunks of the dataset
for chunk in pd.read_csv(file_path, chunksize=chunksize):
    # Concatenate chunks until we reach 1000 rows
    df_5000_rows = pd.concat([df_5000_rows, chunk])
    if len(df_5000_rows) >= 5000:
        # If we have reached or exceeded 1000 rows, break the loop
        df1 = df_5000_rows.iloc[:5000]  # Ensure only the first 1000 rows are kept
        break

In [60]:
df1.head()

,Unnamed: 0,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,0,4.0,True,"03 11, 2014",A240ORQ2LF9LUI,0077613252,{'Format:': ' Loose Leaf'},Michelle W,The materials arrived early and were in excell...,Material Great,1394496000,NaN,NaN
1,1,4.0,True,"02 23, 2014",A1YCCU0YRLS0FE,0077613252,{'Format:': ' Loose Leaf'},Rosalind White Ames,I am really enjoying this book with the worksh...,Health,1393113600,NaN,NaN
2,2,1.0,True,"02 17, 2014",A1BJHRQDYVAY2J,0077613252,{'Format:': ' Loose Leaf'},Allan R. Baker,"IF YOU ARE TAKING THIS CLASS DON""T WASTE YOUR ...",ARE YOU KIDING ME?,1392595200,7.0,NaN
3,3,3.0,True,"02 17, 2014",APRDVZ6QBIQXT,0077613252,{'Format:': ' Loose Leaf'},Lucy,This book was missing pages!!! Important pages...,missing pages!!,1392595200,3.0,NaN
4,4,5.0,False,"10 14, 2013",A2JZTTBSLS1QXV,0077775473,NaN,Albert V.,I have used LearnSmart and can officially say ...,Best study product out there!,1381708800,NaN,NaN


In [61]:
df1.shape

(5000, 13)

In [62]:
df1.columns

Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image'],
      dtype='object')

In [63]:
df1["words"]="default value"
df1["sentences"]="default value"


for i in range(df1.shape[0]):
    df1.at[i,"words"]= list("")
    df1.at[i,"sentences"] = list("")

<ipython-input-63-689a43b2e4ca>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["words"]="default value"
<ipython-input-63-689a43b2e4ca>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["sentences"]="default value"


In [64]:
from nltk.tokenize import sent_tokenize

In [65]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [66]:
for i in range(df1.shape[0]):
    l1= sent_tokenize(str(df1.loc[i,"reviewText"]))
    df1.at[i,"sentences"]=l1

In [67]:
!pip install pywsd

In [68]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [69]:
from pywsd.utils import lemmatize_sentence

In [70]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [71]:
for k in range(df1.shape[0]):
    df1.at[k,"words"]=list("")
    for j in range(len(df1.loc[k,"sentences"])):
        df1.at[k,"words"].extend(lemmatize_sentence(df1.loc[k,"sentences"][j]))


In [72]:
df1["words_sentences"] = "default"

<ipython-input-72-2b56086a37ba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["words_sentences"] = "default"


In [73]:
import functools
for k in range(df1.shape[0]):
    df1.loc[k,"words_sentences"]=functools.reduce(lambda a,b:( str(a)+str(" ")+str(b)),df1.loc[k,"words"])


In [74]:
from sklearn.feature_extraction.text import  CountVectorizer

In [75]:
df2=df1
no_features = 1000
tf_vectorizer = CountVectorizer( max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(df2.words_sentences)


In [76]:
df1_x=pd.DataFrame(tf.A, columns=tf_vectorizer.get_feature_names_out())


In [77]:
df1_y=df1["verified"]

In [78]:
from sklearn.preprocessing import LabelEncoder


In [79]:
le=LabelEncoder()


In [80]:
df1_y_1=pd.DataFrame(df1_y)

In [81]:

df_y_enc=df1_y_1.apply(le.fit_transform)

In [82]:
df_y_enc.columns

Index(['verified'], dtype='object')

In [83]:

df_y_enc.head(5)

,verified
0,1
1,1
2,1
3,1
4,0


**Gradient Boosting Classifier**

In [84]:
from sklearn.ensemble import GradientBoostingClassifier


In [85]:
GBC=GradientBoostingClassifier(n_estimators=3000)

In [86]:
GBC.fit(df1_x,df_y_enc)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(n_estimators=3000)

In [87]:
GBC.score(df1_x,df_y_enc)

0.9848

**Naive Bayes**

In [88]:

from sklearn.naive_bayes import GaussianNB

In [89]:
nv=GaussianNB()

In [90]:
nv.fit(df1_x,df_y_enc)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

In [91]:
nv.score(df1_x,df_y_enc)

0.755

**Random Forest Classifier**

In [92]:
from sklearn.ensemble import RandomForestClassifier


In [93]:
rf=RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model
rf.fit(df1_x,df_y_enc)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(random_state=42)

In [120]:
df1_x_train, df1_x_test, df1_y_train, df1_y_test = train_test_split(df1_x, df1_y, test_size=0.1, random_state=42)

In [121]:
df1_x_train.shape

(4500, 1000)

In [122]:
df1_y_train.shape

(4500,)

In [123]:
rf.fit(df1_x_train,df1_y_train)

RandomForestClassifier(random_state=42)

In [124]:
rf.score(df1_x,df_y_enc)

0.9782